<a href="https://colab.research.google.com/github/piranfar/docking-Analyze/blob/main/Dockinganalyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
import io

def analyze_dlg_with_visualization(save_excel=False, save_plot=False):
    """آنالیز فایل DLG، نمایش جدول و نمودار، و ذخیره اختیاری."""

    # آپلود فایل
    uploaded = files.upload()
    dlg_file_name = next(iter(uploaded))
    dlg_content = uploaded[dlg_file_name].decode('utf-8').splitlines()

    # استخراج نتایج داکینگ
    docking_data = []
    capture_results = False

    for line in dlg_content:
        if "DOCKED: MODEL" in line:
            capture_results = True
        elif "DOCKED: ENDMDL" in line:
            capture_results = False
        if capture_results:
            docking_data.append(line.strip())

    # پردازش امتیازات داکینگ
    binding_energies = []
    rmsd_values = []
    for line in docking_data:
        if "DOCKED: USER    Estimated Free Energy of Binding" in line:
            energy = float(line.split()[-2])
            binding_energies.append(energy)
        if "DOCKED: USER    RMSD from best mode" in line:
            rmsd = float(line.split()[-1])
            rmsd_values.append(rmsd)

    # ایجاد DataFrame
    df_results = pd.DataFrame({
        "Pose": list(range(1, len(binding_energies) + 1)),
        "Binding Energy (kcal/mol)": binding_energies,
        "RMSD (Å)": rmsd_values
    })

    # نمایش جدول
    print("\nDocking Results Table:\n")
    print(df_results)

    # ایجاد نمودار پراکندگی
    plt.figure(figsize=(10, 6))
    plt.scatter(df_results["Pose"], df_results["Binding Energy (kcal/mol)"])
    plt.title("Binding Energy vs. Pose")
    plt.xlabel("Pose")
    plt.ylabel("Binding Energy (kcal/mol)")
    plt.grid(True)
    plt.show()

    # ذخیره در اکسل (اختیاری)
    if save_excel:
        excel_file_name = "docking_results.xlsx"
        df_results.to_excel(excel_file_name, index=False)
        files.download(excel_file_name)

    # ذخیره نمودار به عنوان تصویر (اختیاری)
    if save_plot:
        plot_file_name = "binding_energy_plot.png"
        plt.savefig(plot_file_name)
        files.download(plot_file_name)

# اجرای تابع با ذخیره اختیاری
analyze_dlg_with_visualization(save_excel=True, save_plot=True)

In [1]:
import pandas as pd
from google.colab import files

# Step 1: Upload the AutoDock DLG file
print("Please upload your AutoDock results file (.dlg):")
uploaded = files.upload()

# Get the filename
dlg_filename = list(uploaded.keys())[0]

# Step 2: Read and extract binding energies
binding_energies = []

with open(dlg_filename, "r") as file:
    for line in file:
        if "Estimated Free Energy of Binding" in line:
            try:
                energy = float(line.split()[-2])  # Extract binding energy value
                binding_energies.append(energy)
            except ValueError:
                continue  # Skip lines with incorrect format

# Step 3: Store results in a DataFrame
df_binding_energies = pd.DataFrame({
    "Pose": list(range(1, len(binding_energies) + 1)),
    "Binding Energy (kcal/mol)": binding_energies
})

# Step 4: Display the results
from IPython.display import display
display(df_binding_energies)

# Step 5: Save results as CSV
df_binding_energies.to_csv("binding_energies.csv", index=False)
print("Results saved as 'binding_energies.csv'. You can download it below:")

# Provide download link
from google.colab import files
files.download("binding_energies.csv")


Please upload your AutoDock results file (.dlg):


Saving results.dlg to results.dlg


,Pose,Binding Energy (kcal/mol)


Results saved as 'binding_energies.csv'. You can download it below:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import os
from google.colab import files

# Step 1: Upload the AutoDock DLG file
print("📤 Please upload your AutoDock results file (.dlg):")
uploaded = files.upload()

# Get the filename
dlg_filename = list(uploaded.keys())[0]

# Step 2: Convert the DLG file into readable text format
print("📄 Converting DLG file to text format...")
converted_filename = "converted_results.txt"

with open(dlg_filename, "r", encoding="utf-8", errors="ignore") as infile, open(converted_filename, "w", encoding="utf-8") as outfile:
    for line in infile:
        outfile.write(line)  # Write content to a new text file

print("✅ Conversion complete. Now extracting binding energy values...")

# Step 3: Extract Binding Energies
binding_energies = []

with open(converted_filename, "r", encoding="utf-8") as file:
    for line in file:
        if "Estimated Free Energy of Binding" in line:
            try:
                energy = float(line.split()[-2])  # Extract binding energy value
                binding_energies.append(energy)
            except ValueError:
                continue  # Skip lines with incorrect format

# Step 4: Store results in a DataFrame
df_binding_energies = pd.DataFrame({
    "Pose": list(range(1, len(binding_energies) + 1)),
    "Binding Energy (kcal/mol)": binding_energies
})

# Step 5: Display the extracted data
if df_binding_energies.empty:
    print("⚠️ No binding energies were extracted. Please check your DLG file format.")
else:
    from IPython.display import display
    display(df_binding_energies)

    # Save as CSV
    csv_filename = "binding_energies.csv"
    df_binding_energies.to_csv(csv_filename, index=False)
    print(f"✅ Results saved as '{csv_filename}'. You can download it below:")

    # Provide download link
    files.download(csv_filename)


📤 Please upload your AutoDock results file (.dlg):


Saving grid1.dlg to grid1.dlg
📄 Converting DLG file to text format...
✅ Conversion complete. Now extracting binding energy values...
⚠️ No binding energies were extracted. Please check your DLG file format.
